In [ ]:
!pip install google-generativeai "llama-index>=0.10" streamlit faiss-cpu python-docx tiktoken PyMuPDF pyngrok -q
# !pip install pandas

In [5]:
from google.colab import drive
import os
from google.colab import userdata
import google.generativeai as genai

drive.mount('/content/drive')
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks/MR_Chatbot' # Choose your project directory name
os.chdir(PROJECT_DIR) # Navigate to your project directory (optional, but good for relative paths later)
print(f"Current working directory: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory: /content/drive/MyDrive/Colab Notebooks/MR_Chatbot


In [6]:
# --- Retrieve API key: Gemini version ---
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if GEMINI_API_KEY is None:
        raise ValueError("Gemini API key not found in Colab Secrets. Please ensure it's added correctly with the name GEMINI_API_KEY.")
    genai.configure(api_key=GEMINI_API_KEY)
    print("Gemini API Key configured successfully.")
except Exception as e:
    print(f"Error configuring Gemini API Key: {e}")
    print("Please ensure you have added your GEMINI_API_KEY to Colab Secrets (Key icon on the left).")

Gemini API Key configured successfully.


In [7]:
if 'GEMINI_API_KEY' in locals() and GEMINI_API_KEY: # Check if API key was loaded and configured
    try:
        # For text-only input, use a Gemini model like "gemini-1.5-flash-latest" or "gemini-pro"
        # model = genai.GenerativeModel('gemini-pro')
        model = genai.GenerativeModel('gemini-1.5-flash-latest') # Often good for quick tasks and cost-effective

        prompt = "What are three emerging trends in sustainable packaging for 2025, according to your knowledge cutoff?"
        response = model.generate_content(prompt)

        print("API Call Successful!")
        print("Response from Gemini:")
        # print(response.text) # Access the text directly
        # Or, to handle potential blocks if any (though for simple text usually not an issue)
        if response.parts:
            print(response.parts[0].text)
        else:
            print("No content parts found in response. Full response:", response)


    except Exception as e:
        print(f"An error occurred during the Gemini API call: {e}")
        # You can print response.prompt_feedback for safety ratings if the call fails due to content policy
        # For example, if hasattr(response, 'prompt_feedback'): print(response.prompt_feedback)
else:
    print("Gemini API Key not set or configured. Please complete Step 1.2 correctly.")

API Call Successful!
Response from Gemini:
My knowledge cutoff is 2021, so I can't predict specific emerging trends *for* 2025. However, based on the trends observable in 2021, I can extrapolate three likely emerging trends in sustainable packaging by then:

1. **Increased use of bio-based and compostable materials:**  The movement away from petroleum-based plastics was already accelerating in 2021. By 2025, we would likely see a wider adoption and refinement of bioplastics derived from renewable sources like seaweed, mushrooms, and agricultural waste.  Improved compostability and biodegradability, alongside reduced cost and improved performance characteristics, would be key factors driving this trend.

2. **Focus on packaging reduction and reuse systems:**  Minimizing packaging material would continue to be a priority.  This would manifest in lighter-weight packaging designs, innovative packaging formats that reduce material use, and a strong push towards reusable packaging systems, i

In [9]:
# Ensure PROJECT_DIR is set from Cell 1.0 or define it here if running independently

sub_directories = ["data", "notebooks", "vectorstore", "utils", "app"] # 'app' for Streamlit

# Create a placeholder requirements.txt (we'll populate it later for deployment)
requirements_file = os.path.join(PROJECT_DIR, "requirements.txt")
if not os.path.exists(requirements_file):
    with open(requirements_file, "w") as f:
        f.write("# Google Generative AI and LlamaIndex\n")
        f.write("google-generativeai\n")
        f.write("llama-index>=0.10\n")
        f.write("faiss-cpu\n")
        f.write("# Document Loaders\n")
        f.write("pandas\n")
        f.write("python-docx\n")
        f.write("PyMuPDF\n")
        f.write("# Web App\n")
        f.write("streamlit\n")
        f.write("# Other\n")
        f.write("tiktoken\n") # LlamaIndex might still use this for some operations
    print(f"'{requirements_file}' created.")
else:
    print(f"'{requirements_file}' already exists.")

# Create a placeholder .gitignore (useful if you use Git)
gitignore_file = os.path.join(PROJECT_DIR, ".gitignore")
if not os.path.exists(gitignore_file):
    with open(gitignore_file, "w") as f:
        f.write("# Environment files\n")
        f.write(".env\n")
        f.write("*.env\n")
        f.write("\n# Colab/Jupyter specific\n")
        f.write(".ipynb_checkpoints/\n")
        f.write("__pycache__/\n")
        f.write("\n# Data files (if large or sensitive and you don't want to commit them)\n")
        f.write("# data/\n") # Example: uncomment to ignore data folder
        f.write("\n# Vectorstore files\n")
        f.write("vectorstore/\n") # Usually good to ignore the built index
    print(f"'{gitignore_file}' created.")
else:
    print(f"'{gitignore_file}' already exists.")

# Create a placeholder README.md
readme_file = os.path.join(PROJECT_DIR, "README.md")
if not os.path.exists(readme_file):
    with open(readme_file, "w") as f:
        f.write("# My Market AI Assistant Project (using Gemini)\n\n")
        f.write("This project aims to build an AI assistant for market research insights, powered by Google Gemini.\n")
    print(f"'{readme_file}' created.")
else:
    print(f"'{readme_file}' already exists.")

Directory '/content/drive/MyDrive/Colab Notebooks/MR_Chatbot/data' already exists.
Directory '/content/drive/MyDrive/Colab Notebooks/MR_Chatbot/notebooks' already exists.
Directory '/content/drive/MyDrive/Colab Notebooks/MR_Chatbot/vectorstore' already exists.
Directory '/content/drive/MyDrive/Colab Notebooks/MR_Chatbot/utils' already exists.
Directory '/content/drive/MyDrive/Colab Notebooks/MR_Chatbot/app' already exists.
'/content/drive/MyDrive/Colab Notebooks/MR_Chatbot/requirements.txt' already exists.
'/content/drive/MyDrive/Colab Notebooks/MR_Chatbot/.gitignore' already exists.
'/content/drive/MyDrive/Colab Notebooks/MR_Chatbot/README.md' already exists.


In [ ]:
# Cell 1.5 (Run this only if you want to use Git)

# Configure Git (only need to do this once per Colab session if not persisted)
!git config --global user.name "stimmarine"
!git config --global user.email "stevenyap2357@gmail.com"

# Navigate to your project directory (ensure this is correct)
os.chdir(PROJECT_DIR) # PROJECT_DIR should be set from Cell 1.0
print(f"Current directory for Git operations: {os.getcwd()}")

# Initialize Git, add files, commit, and push
!git init
!git add .
!git commit -m "Phase 1: Initial project setup with Gemini and Colab environment"
!git remote add origin https://github.com/stimmarine/MR_Chatbot.git
!git branch -M main
!git push -u origin main

print("Git commands executed. Replace THE_URL_OF_YOUR_NEW_EMPTY_REPO and uncomment the lines above to push.")
print("If you push, you might be prompted for your Git credentials.")